In [1]:
import polars as pl

train_data = pl.read_csv(
    "/Users/shinichiro.saito/automated_essay_scoring/data/train.csv"
)

persuade_data = pl.read_csv(
    "/Users/shinichiro.saito/automated_essay_scoring/data/persuade_2.0_human_scores_demo_id_github.csv"
)

In [3]:
assignment_list = list(set(persuade_data["assignment"]))
num_list = list(range(15))
assignment_num_dict = dict(zip(assignment_list, num_list))
assignment_num_dict

{'Your principal has decided that all students must participate in at least one extracurricular activity. For example, students could participate in sports, work on the yearbook, or serve on the student council. Do you agree or disagree with this decision? Use specific details and examples to convince others to support your position. ': 0,
 'Some schools offer distance learning as an option for students to attend classes from home by way of online or video conferencing. Do you think students would benefit from being able to attend classes from home? Take a position on this issue. Support your response with reasons and examples.': 1,
 'Some schools require students to complete summer projects to assure they continue learning during their break. Should these summer projects be teacher-designed or student-designed? Take a position on this question. Support your response with reasons and specific examples.': 2,
 "You have read the article 'Unmasking the Face on Mars.' Imagine you are a sci

In [4]:
train_data

essay_id,full_text,score
str,str,i64
"""000d118""","""Many people have car where the…",3
"""000fe60""","""I am a scientist at NASA that …",3
"""001ab80""","""People always wish they had th…",4
"""001bdc0""","""We all heard about Venus, the …",4
"""002ba53""","""Dear, State Senator This is a…",3
…,…,…
"""ffd378d""","""the story "" The Challenge of E…",2
"""ffddf1f""","""Technology has changed a lot o…",4
"""fff016d""","""If you don't like sitting arou…",2


In [5]:
len(set(persuade_data["full_text"]) & set(train_data["full_text"]))

12871

In [6]:
# 学習データに含まれているかどうかをbool値を付与する
persuade_data = persuade_data.with_columns(
    pl.col("full_text").is_in(list(train_data["full_text"])).alias("is_train_contains")
)

In [7]:
# Kaggleデータにも存在するpromptを出力
set(persuade_data.filter(pl.col("is_train_contains")).get_column("assignment"))

{'In "The Challenge of Exploring Venus," the author suggests studying Venus is a worthy pursuit despite the dangers it presents. Using details from the article, write an essay evaluating how well the author supports this idea. Be sure to include: a claim that evaluates how well the author supports the idea that studying Venus is a worthy pursuit despite the dangers; an explanation of the evidence from the article that supports your claim; an introduction, a body, and a conclusion to your essay.',
 'In the article "Making Mona Lisa Smile," the author describes how a new technology called the Facial Action Coding System enables computers to identify human emotions. Using details from the article, write an essay arguing whether the use of this technology to read the emotional expressions of students in a classroom is valuable.',
 'In the article “Driverless Cars are Coming,” the author presents both positive and negative aspects of driverless cars. Using details from the article, create a

In [8]:
# assignment_num_dictを用いて、assignmentに数値に変換する
persuade_data = persuade_data.with_columns(
    pl.col("assignment").replace(assignment_num_dict).alias("assignment_num")
)

In [18]:
persuade_data.filter(pl.col("is_train_contains"))["prompt_name"].n_unique()

7

In [19]:
persuade_data.filter(pl.col("is_train_contains")).height

12871

In [20]:
persuade_data.filter(~pl.col("is_train_contains"))["prompt_name"].n_unique()

9

In [21]:
persuade_data.filter(~pl.col("is_train_contains"))["prompt_name"].value_counts()

prompt_name,count
str,u32
"""Distance learning""",2157
"""Summer projects""",1750
"""Grades for extracurricular act…",1626
"""Cell phones at school""",1656
"""Phones and driving""",1168
"""Mandatory extracurricular acti…",1670
"""Seeking multiple opinions""",1552
"""Community service""",1542
"""Car-free cities""",4


In [10]:
persuade_data["prompt_name"].value_counts()

prompt_name,count
str,u32
"""Cell phones at school""",1656
"""Distance learning""",2157
"""Facial action coding system""",2167
"""Grades for extracurricular act…",1626
"""Community service""",1542
…,…
"""""A Cowboy Who Rode the Waves""""",1372
"""Seeking multiple opinions""",1552
"""Car-free cities""",1959


In [53]:
persuade_data.write_csv(
    "/Users/shinichiro.saito/automated_essay_scoring/data/persuade_w_is_tr_con_as_num.csv"
)

In [55]:
persuade_data.head()

essay_id_comp,full_text,holistic_essay_score,word_count,prompt_name,task,assignment,source_text,gender,grade_level,ell_status,race_ethnicity,economically_disadvantaged,student_disability_status,is_train_contains,assignment_num
str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,bool,str
"""423A1CA112E2""","""Phones Modern humans today ar…",3,378,"""Phones and driving""","""Independent""","""Today the majority of humans o…",null,"""M""",null,null,"""Black/African American""",null,null,false,"""11"""
"""BC75783F96E3""","""This essay will explain if dri…",4,432,"""Phones and driving""","""Independent""","""Today the majority of humans o…",null,"""M""",null,null,"""Black/African American""",null,null,false,"""11"""
"""74C8BC7417DE""","""Driving while the use of cellu…",2,179,"""Phones and driving""","""Independent""","""Today the majority of humans o…",null,"""F""",null,null,"""White""",null,null,false,"""11"""
"""A8445CABFECE""","""Phones & Driving Drivers shou…",3,221,"""Phones and driving""","""Independent""","""Today the majority of humans o…",null,"""M""",null,null,"""Black/African American""",null,null,false,"""11"""
"""6B4F7A0165B9""","""Cell Phone Operation While Dri…",4,334,"""Phones and driving""","""Independent""","""Today the majority of humans o…",null,"""M""",null,null,"""White""",null,null,false,"""11"""


In [59]:
from collections import Counter

Counter(list(persuade_data.filter(pl.col("is_train_contains"))["assignment_num"]))

Counter({'4': 2167,
         '8': 2046,
         '5': 1955,
         '2': 1886,
         '9': 1862,
         '0': 1583,
         '10': 1372})